In [58]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain ragas datasets cohere

^C


In [1]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import os

c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")
OPENAI_MODEL_GPT4 = os.environ.get("OPENAI_MODEL_GPT4")
OPENAI_DEPLOYMENT_GPT4 = os.environ.get("OPENAI_DEPLOYMENT_GPT4")

# Evaluation part

### Ground truth

In [3]:
questions = [
    "Who was Alexei Navalny supposed to be exchanged for in the planned prisoner swap, according to his colleague Maria Pevchikh?",
    "Why did the door blow out of Boeing 737 Max shortly after take-off?",
    "When did it mark 2 years since Russia's envasion in Ukraine?",
    "What is Sora AI, how does it work, and when can the general public expect to use it?",
    "What are the potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump, and how might they impact the outcome?",
    "Why does Elon Musk want a bigger stake in Tesla, and what concerns does he express about the current structure of the company?",
    "What are the shopping habits of Gen Alpha, and why are mature brands interested in targeting this demographic?",
    "What happened during the Hamas attacks on Israel in 2023?",
    "On January 15, 2024, how did Kim Jong Un call South Korea?",
    "Why did Prince William pull out of the godfather's memorial service on 27.02.2024?"
]

ground_truths = [
    ["Alexei Navalny was supposed to be exchanged for Vadim Krasikov, a Russian hitman serving a life sentence for murder in Germany, as claimed by Maria Pevchikh."],  
    ["Bolts were missing."],
    ["On February 24, 2024."],
    ["Sora AI is an artificial intelligence tool developed by OpenAI that can generate realistic videos up to 1 minute long based on textual prompts. It utilizes diffusion models, a method used in AI image generators, to create videos by understanding the association between images and accompanying alt text. As of now, Sora is not available to the general public; OpenAI is following a cautious approach by first testing it with a small group of 'red teamers' for potential harm or risks, followed by availability to visual artists, designers, and filmmakers before a potential public release, likely under a pay-to-use model similar to GPT. While Sora has shown significant advancements in AI video generation compared to previous attempts, it is not perfect, with occasional flaws and limitations seen in hand-picked videos released by OpenAI."],
    ["The potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump include contrasting economic views, with Americans historically voting based on their economic well-being; the contentious topics of abortion and immigration, with Democrats focusing on abortion rights and Republicans emphasizing border security; and external factors like the Gaza War and its impact on Biden's support base. Other factors include crime rates, environmental concerns, and foreign policy. Additionally, the health and age of both candidates could be scrutinized, and the potential emergence of third-party candidates or independents might introduce unpredictability. Trump's legal challenges, facing 91 charges and four criminal trials, could influence public opinion, and the specter of the January 2021 Capitol attack may resonate differently among Republican and Democratic voters. Overall, these factors contribute to the complexity and uncertainty of the upcoming presidential race."],
    ["Elon Musk wants a bigger stake in Tesla to gain more control over the company's direction, especially regarding its investments in artificial intelligence (AI) features. He is concerned about Tesla's vulnerability to a 'takeover by dubious interests' and aims to have 25% voting control to ensure the company's leadership in AI and robotics. Musk suggests that without this control, he would prefer to develop products outside of Tesla."],
    ["Gen Alpha, born between 2010 and 2024, prefers to shop at adult brands like Lululemon, Sephora, Walmart, and Target, following the trends of their millennial parents. Adult brands are interested in targeting Gen Alpha due to their economic potential, with an estimated economic footprint of $5.46 trillion by 2029. These brands can secure the loyalty of the next generation by expanding their offerings to cater to the specific needs of Gen Alpha."],
    ["On the morning of 7 October, waves of Hamas gunmen stormed across Gaza's border into Israel, killing about 1,200 people. Hamas also fired thousands of rockets. Those killed included children, the elderly and 364 young people at a music festival.  Hamas took more than 250 others to Gaza as hostages."],
    ["Kim Jong Un  'principal enemy.' He intensified nuclear threats, conducted missile tests, and abolished government agencies promoting cooperation and reunification. Analysts, including Robert L. Carlin and Siegfried Hecker, suggest he may be preparing for a military attack on South Korea."],
    ["Due to 'personal matter'."]
]
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""],[""],[""]]

In [4]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)
llm_gpt4 = AzureChatOpenAI(model_name=OPENAI_MODEL_GPT4, azure_deployment=OPENAI_DEPLOYMENT_GPT4,temperature=0)

In [6]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
    )
    return result

In [7]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False,
    )
    return result

### General answers by LLM

In [8]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [9]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)
llm_chain_gpt4 =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm_gpt4}
)

In [10]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

In [11]:
llm_results = evaluation_llm(questions, answers_llm, contexts_llm, ground_truths)
print(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 40/40 [00:06<00:00,  5.78it/s]


{'faithfulness': 1.0000, 'answer_relevancy': 0.3756, 'answer_similarity': 0.8477, 'answer_correctness': 0.4599}


In [ ]:
answers_llm_gpt4 = []
for query in questions:
    response = llm_chain_gpt4.invoke({"question": query})
    answers_llm_gpt4.append(response["response"].content)
llm_results_gpt4 = evaluation_llm(questions, answers_llm_gpt4, contexts_llm, ground_truths)
print(llm_results_gpt4)

### Naive RAG

In [12]:
loader = DirectoryLoader('../news', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [13]:
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db_naive = Chroma.from_documents(chunks, embeddings_client, persist_directory = "../vectordb/naive")
retriever_naive = db_naive.as_retriever()

Created a chunk of size 4589, which is longer than the specified 4000
Created a chunk of size 4082, which is longer than the specified 4000


In [15]:
template = """User input {question}. 
Context {context}."""
prompt = ChatPromptTemplate.from_template(template)

In [16]:
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever_naive, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [17]:
answers_naive = []
contexts_naive = []
for query in questions:
    try:  
        response = retrieval_augmented_qa_chain.invoke({"question": query})
        # Access the response content
        answers_naive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_naive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_naive.append("No answer")
        context_full = retriever_naive.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_naive.append(context_content)

In [18]:
result_naive_rag = evaluation_rag(questions, answers_naive, contexts_naive, ground_truths)
print(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.56it/s]


{'faithfulness': 0.8696, 'answer_relevancy': 0.6844, 'context_precision': 0.9000, 'context_recall': 0.9800, 'answer_similarity': 0.9033, 'answer_correctness': 0.5487}


In [ ]:
# df = result.to_pandas()
# df

## try recursive text splitter

In [19]:
text_splitter = text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()
chunks_r = text_splitter.split_documents(documents)
db_basic = Chroma.from_documents(chunks_r, embeddings_client, persist_directory = "../vectordb/recursive_basic")
retriever_basic = db_basic.as_retriever()

In [20]:
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever_basic, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [21]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    try:  
        response = retrieval_augmented_qa_chain.invoke({"question": query})
        # Access the response content
        answers_recursive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_recursive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_recursive.append("No answer")
        context_full = retriever_basic.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_recursive.append(context_content)

In [22]:
result_recursive = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
print(result_recursive)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 58, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 91, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDriv

{'faithfulness': 0.7883, 'answer_relevancy': 0.8552, 'context_precision': 0.8919, 'context_recall': 0.5964, 'answer_similarity': 0.9500, 'answer_correctness': 0.7503}


## chunk size change

In [23]:
def change_chunk_size(retriever):
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    answers_recursive = []
    contexts_recursive = []

    for query in questions:
        try:  
            response = retrieval_augmented_qa_chain.invoke({"question": query})
            # Access the response content
            answers_recursive.append(response["response"].content)
            # Access the context content
            context_content = [context.page_content for context in response["context"]]
            contexts_recursive.append(context_content)  
        except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_recursive.append("No answer")
            context_full = retriever.get_relevant_documents(query)
            context_content = [context.page_content for context in context_full]
            contexts_recursive.append(context_content)


    result = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
    return result

In [24]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
chunks_1000 = text_splitter.split_documents(documents)
print(len(chunks_1000))
db_1000 = Chroma.from_documents(chunks_1000, embeddings_client, persist_directory = "../vectordb/recursive_1000")
retriever_1000 = db_1000.as_retriever()
result_1000 = change_chunk_size(retriever_1000)
print("CHUNK SIZE 1000")
print(result_1000)

56


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:11<00:00,  5.14it/s]


CHUNK SIZE 1000
{'faithfulness': 1.0000, 'answer_relevancy': 0.6966, 'context_precision': 0.9333, 'context_recall': 0.9800, 'answer_similarity': 0.9226, 'answer_correctness': 0.6393}


In [25]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 50)
chunks_500 = text_splitter.split_documents(documents)
print(len(chunks_500))
db_500 = Chroma.from_documents(chunks_500, embeddings_client, persist_directory = "../vectordb/recursive_500")
retriever_500 = db_500.as_retriever()
result_500 = change_chunk_size(retriever_500)
print("CHUNK SIZE 500")
print(result_500)

107


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:08<00:00,  7.23it/s]


CHUNK SIZE 500
{'faithfulness': 0.8750, 'answer_relevancy': 0.8328, 'context_precision': 0.8500, 'context_recall': 0.9500, 'answer_similarity': 0.9173, 'answer_correctness': 0.6245}


In [26]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
chunks_2000 = text_splitter.split_documents(documents)
print(len(chunks_2000))
db_2000 = Chroma.from_documents(chunks_2000, embeddings_client, persist_directory = "../vectordb/recursive_2000")
retriever_2000 = db_2000.as_retriever()
result_2000 = change_chunk_size(retriever_2000)
print("CHUNK SIZE 2000")
print(result_2000)

37


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:08<00:00,  7.21it/s]


CHUNK SIZE 2000
{'faithfulness': 1.0000, 'answer_relevancy': 0.7395, 'context_precision': 0.9500, 'context_recall': 0.7750, 'answer_similarity': 0.9190, 'answer_correctness': 0.6177}


In [27]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
chunks_3000 = text_splitter.split_documents(documents)
print(len(chunks_3000))
db_3000 = Chroma.from_documents(chunks_3000, embeddings_client, persist_directory = "../vectordb/recursive_3000")
retriever_3000 = db_3000.as_retriever()
result_3000 = change_chunk_size(retriever_3000)
print("CHUNK SIZE 3000")
print(result_3000)

32


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  18%|█▊        | 11/60 [00:02<00:07,  6.74it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 58, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 91, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sigitalapin

CHUNK SIZE 3000
{'faithfulness': 0.7327, 'answer_relevancy': 0.9200, 'context_precision': 0.6936, 'context_recall': 0.8889, 'answer_similarity': 0.8734, 'answer_correctness': 0.7756}


### now time to look for different top-k

Note: We continue with the size chunk of 1000 as it had the highest average score

In [28]:
retriever_2 = db_1000.as_retriever(search_kwargs={"k": 2})
result_2 = change_chunk_size(retriever_2)
print("CHUNK SIZE 1000, K=2")
print(result_2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:08<00:00,  6.88it/s]


CHUNK SIZE 1000, K=2
{'faithfulness': 1.0000, 'answer_relevancy': 0.7317, 'context_precision': 0.9000, 'context_recall': 1.0000, 'answer_similarity': 0.9071, 'answer_correctness': 0.5908}


In [29]:
retriever_3 = db_1000.as_retriever(search_kwargs={"k": 3})
result_3 = change_chunk_size(retriever_3)
print("CHUNK SIZE 1000, K=3")
print(result_3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:09<00:00,  6.33it/s]


CHUNK SIZE 1000, K=3
{'faithfulness': 1.0000, 'answer_relevancy': 0.7832, 'context_precision': 0.9333, 'context_recall': 0.9250, 'answer_similarity': 0.9326, 'answer_correctness': 0.6117}


In [30]:
retriever_5 = db_1000.as_retriever(search_kwargs={"k": 5})
result_5 = change_chunk_size(retriever_5)
print("CHUNK SIZE 1000, K=5")
print(result_5)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:14<00:00,  4.03it/s]


CHUNK SIZE 1000, K=5
{'faithfulness': 0.9028, 'answer_relevancy': 0.6788, 'context_precision': 0.9333, 'context_recall': 1.0000, 'answer_similarity': 0.9250, 'answer_correctness': 0.6024}


### look for different retrievers

4 chunks was the best score

#### parent document retriever

In [31]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000)
child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents",persist_directory = "../vectordb/parent", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)
parent_document_retriever.add_documents(documents)
result_parent = change_chunk_size(parent_document_retriever)
print(result_parent)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:09<00:00,  6.61it/s]


{'faithfulness': 1.0000, 'answer_relevancy': 0.8276, 'context_precision': 0.8500, 'context_recall': 0.9667, 'answer_similarity': 0.9079, 'answer_correctness': 0.5174}


In [39]:
parent_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap = 50)
child_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_small",persist_directory = "../vectordb/parent_small", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_small = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_small,
    parent_splitter=parent_splitter_small,
)
parent_document_retriever_small.add_documents(documents)
result_parent_small = change_chunk_size(parent_document_retriever_small)
print(result_parent_small)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-1106' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py", line 2018, in aclose
    await self._transport.aclose()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_transports\default.py", line 385, in aclose
    await self._pool.aclose()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-pack

{'faithfulness': 0.6965, 'answer_relevancy': 0.7556, 'context_precision': 0.9548, 'context_recall': 0.9833, 'answer_similarity': 0.9619, 'answer_correctness': 0.8100}


In [41]:
parent_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500)
child_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_large",persist_directory = "../vectordb/parent_large", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_large = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_large,
    parent_splitter=parent_splitter_large,
)
parent_document_retriever_large.add_documents(documents)
result_parent_large = change_chunk_size(parent_document_retriever_large)
print(result_parent_large)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  30%|███       | 18/60 [00:04<00:08,  4.69it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 58, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 91, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sigitalapin

{'faithfulness': 0.7277, 'answer_relevancy': 0.6453, 'context_precision': 0.9647, 'context_recall': 0.8417, 'answer_similarity': 0.8456, 'answer_correctness': 0.6383}


#### Maximum marginal relevance retrieval

In [32]:
retriever_mmr = db_1000.as_retriever(search_type="mmr",search_kwargs={"k": 3})
result_mmr = change_chunk_size(retriever_mmr)
print("Marginal relevance")
print(result_mmr)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 58, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 91, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDriv

Marginal relevance
{'faithfulness': 0.9599, 'answer_relevancy': 0.7523, 'context_precision': 0.8724, 'context_recall': 0.8370, 'answer_similarity': 0.8706, 'answer_correctness': 0.8213}


#### BM25

In [33]:
from langchain.retrievers import BM25Retriever

retriever_bm25 = BM25Retriever.from_documents(chunks_1000)
result_bm25 = change_chunk_size(retriever_bm25)
print("Marginal relevance")
print(result_bm25)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.81it/s]


Marginal relevance
{'faithfulness': 0.9792, 'answer_relevancy': 0.7536, 'context_precision': 0.9000, 'context_recall': 0.9229, 'answer_similarity': 0.9190, 'answer_correctness': 0.5862}


#### Ensambler - Hybrid

In [34]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever_1 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.75, 0.25])
result_ensemble1 = change_chunk_size(ensemble_retriever_1)
print("Ensambler")
print(result_ensemble1)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.99it/s]


Ensambler
{'faithfulness': 1.0000, 'answer_relevancy': 0.7810, 'context_precision': 0.9133, 'context_recall': 0.9800, 'answer_similarity': 0.9233, 'answer_correctness': 0.6444}


In [35]:
ensemble_retriever_2 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.5, 0.5])
result_ensemble2 = change_chunk_size(ensemble_retriever_2)
print("Ensambler")
print(result_ensemble2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:08<00:00,  7.11it/s]


Ensambler
{'faithfulness': 1.0000, 'answer_relevancy': 0.8570, 'context_precision': 0.9367, 'context_recall': 0.9800, 'answer_similarity': 0.9225, 'answer_correctness': 0.6061}


In [36]:
ensemble_retriever_3 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.25,0.75])
result_ensemble3 = change_chunk_size(ensemble_retriever_3)
print("Ensambler")
print(result_ensemble3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:09<00:00,  6.49it/s]


Ensambler
{'faithfulness': 1.0000, 'answer_relevancy': 0.6792, 'context_precision': 0.9333, 'context_recall': 0.9800, 'answer_similarity': 0.9186, 'answer_correctness': 0.6378}


#### Multi-stage - reranker

In [ ]:
%pip install --upgrade --quiet  cohere

Note: you may need to restart the kernel to use updated packages.


In [42]:
import getpass

os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [46]:
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
retriever_context = EnsembleRetriever(retrievers=[retriever_bm25, retriever_5], weights=[0.5, 0.5])
compressor = CohereRerank(top_n = 3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context
)

result_compression = change_chunk_size(compression_retriever)
print("Reranker")
print(result_compression)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  32%|███▏      | 19/60 [00:04<00:07,  5.37it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 58, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 91, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sigitalapin

Reranker
{'faithfulness': 0.6801, 'answer_relevancy': 0.7920, 'context_precision': 0.7477, 'context_recall': 0.9250, 'answer_similarity': 0.9693, 'answer_correctness': 0.8088}


#### creating context by remaking the query

In [45]:
template_context = "Generate a search query to fetch the relevant documents using the user's {question}. Craft a query that specifically targets the keywords in the question. In the answer provide only the query."
prompt_context = ChatPromptTemplate.from_template(template_context)

In [47]:
answers_final = []
contexts_final = []
# retriever_context_q = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.5, 0.5])
llm_for_context =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm}
)
for query in questions:
    response_check = llm_for_context.invoke({"question": query})
    search_query = response_check["response"].content
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
)
    docs = ensemble_retriever_2.get_relevant_documents(search_query)
    formatted_docs = []
    for doc in docs:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final.append(response["response"].content)
            contexts_final.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final.append("No answer")
            contexts_final.append(formatted_docs)


result_search_query = evaluation_rag(questions, answers_final, contexts_final, ground_truths)
print(result_search_query)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   2%|▏         | 1/60 [00:02<02:11,  2.24s/it]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 58, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 91, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sigitalapina

{'faithfulness': 0.7342, 'answer_relevancy': 0.7500, 'context_precision': 0.8029, 'context_recall': 0.8111, 'answer_similarity': 0.7596, 'answer_correctness': 0.8017}


### change model to GPT-4

In [48]:
llm_gpt4 = AzureChatOpenAI(model_name=OPENAI_MODEL_GPT4, azure_deployment=OPENAI_DEPLOYMENT_GPT4,temperature=0)

In [49]:
retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | ensemble_retriever_2, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm_gpt4, "context": itemgetter("context")}
    )
answers_gpt4 = []
contexts_gpt4 = []

for query in questions:
    try:  
        response = retrieval_augmented_qa_chain.invoke({"question": query})
        # Access the response content
        answers_gpt4.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_gpt4.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_gpt4.append("No answer")
        context_full = retriever_500.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_gpt4.append(context_content)

result_gpt4 = evaluation_rag(questions, answers_gpt4, contexts_gpt4, ground_truths)
print(result_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.76it/s]


{'faithfulness': 0.9611, 'answer_relevancy': 0.7860, 'context_precision': 0.9367, 'context_recall': 0.9800, 'answer_similarity': 0.9264, 'answer_correctness': 0.7303}
